### [Index of /reimers/sentence-transformers/v0.2/](https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/)

In [1]:
# import os
# default is in ~/.cache
# os.environ['HF_HOME'] = '/home/jovyan/cache/'
# os.environ['TOKENIZERS_PARALLELISM'] = 'true'

### reload from cache snapshot (need to locate the config.json location)

In [ ]:
# MODEL='thenlper/gte-small'
# MODEL='/home/jovyan/cache/hub/models--thenlper--gte-small/snapshots/17e1f347d17fe144873b1201da91788898c639cd'
# gte_model = SentenceTransformer(MODEL)

In [1]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

In [3]:
model = SentenceTransformer('gte-small')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 25, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
# model = SentenceTransformer('all-MiniLM-L6-v2')
# model

In [ ]:

# MODEL = 'all-mpneimport os
# os.environ['HF_HOME'] = '/blabla/cache/'t-base-v2'
# MODEL = r'../all-MiniLM-L6-v2'
MODEL='thenlper/gte-small'
# MODEL = 'all-MiniLM-L12-v2


################# LOAD SENTENCE TRANSFORMER MODEL ###########################
# Load the embedding model and tokenizer manually

word_embedding_model = models.Transformer(MODEL)

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# Assemble the sentence transformer model
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

MAX_SEQ_LENGTH = 25

model[0].max_seq_length = MAX_SEQ_LENGTH
model[0].tokenizer.max_seq_length = MAX_SEQ_LENGTH
model[0].do_lower_case = True
model[1].pooling_mode_cls_token=True
model[1].pooling_mode_mean_tokens=False
model[1].pooling_mode_max_tokens=False
model[1].pooling_mode_mean_sqrt_len_tokens=False
model[1].pooling_mode_weightedmean_tokens=False
model[1].pooling_mode_lasttoken=False

print(f"{model=}")

sentences = ["This is an example sentence", "Each sentence is converted"]

# model = SentenceTransformer('../all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)
# Save the model locally
model.save(r'../gte-small')

### Using Transformers Library

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

sentences = ["This is an example sentence", "Each sentence is converted"]

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


tokenizer = AutoTokenizer.from_pretrained(r',,/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained(r'../all-MiniLM-L6-v2')

encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

print(f"{encoded_input=}")
with torch.no_grad():
    model_output = model(**encoded_input)
    print(f"{model_output['last_hidden_state'].shape=}")
    print(f"{model_output['pooler_output'].shape=}")
# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
print(f"{sentence_embeddings.shape=}")
print("Sentence embeddings:")
print(sentence_embeddings)